# **Team Name:** [The name of your team you choose]
# **Author:** `[Write in the names of the authors of this notebook.]`

# Table of Contents

**Note:** Use the **table of contents sidebar** on the left (click the ☰ list icon) to navigate between sections.

Below is an outline of the resources contained in this document:

## Getting Started

- FA25 Dataset Overview
- Defining Functions

## Query Functions

- Query Cell Metadata
- Query Temperature Metadata
- Query Wavelength Metadata
- Query Pixel Number
- Query All Measurement Data for a Cell
- Query Measurement Data with a Common Date for a Cell

## Data Retrieval

- Getting Data from a single Cell/Pixel/Date
- Getting Data from Multiple Cells/Pixels/Dates

## Analysis & Visualization

- Calculate EQE for a given cell/pixel/date
- Plot EQE over time
- Plot IV Data
- Calculate PCE and Related Quantities

## Introduction to HDF5 Files

---

# FA25 Dataset Overview

**FA25 Dataset Info:** Cells 125-197 (3-digit) | 8 pixels | 55°C, White light | 0.14 cm² area | `data_fa25.h5`

**Note:** Stress wavelength is stored as `'White'` (not `'White light'`) in the database.

## Quick Start

This notebook contains helper functions for analyzing solar cell degradation data from the Fall 2025 semester. The data is stored in an HDF5 file (`data_fa25.h5`) which efficiently organizes measurements for 57 cells over multiple measurement dates.

## Key Information

- **Cell Numbers**: 125-197 (3-digit format)
- **Pixels per Cell**: 8 pixels (numbered 1-8)
- **Stress Conditions**: All cells stressed at 55°C with white light
- **Measurement Period**: September-October 2025
- **Unmasked Pixel Area**: 0.14 cm²

## Measurement Types

For each cell and pixel, we collect three types of measurements:

1. **Power Data** - Measured once per cell (wavelength vs power)
2. **Current Data** - Measured for each pixel (wavelength vs current)
3. **IV Data** - Measured for each pixel (voltage vs current, forward/reverse scans)

## How to Use This Notebook

1. **Execute the function definitions** in the "Defining Functions" section below
2. **Use the query functions** to find cells with specific properties
3. **Retrieve data** for analysis using the get_data functions
4. **Calculate metrics** like EQE and PCE using the provided functions
5. **Visualize results** with the plotting functions

For more details about the HDF5 file format and structure, see Introduction to HDF5 Files section.

# Defining Functions

The functions in the cell below will assist you in retrieving the information you need from the HDF5 file. These functions are designed to help you access specific datasets, filter data based on metadata, and calculate metrics directly from the data stored in the HDF5 file.

You should not need to make any edits to the functions in this cell.

### **Simply execute the cell below to define the functions, HIDE THE CELL OUTPUT, and move on to using them in your analysis.**

However, we have provided detailed comments within the functions to help you understand what each function is doing and how it works. Feel free to review the comments if you are interested in learning more about the implementation.

If you want to access or run only a portion of the code included here, below is a list of the functions included:

1. **read_metadata** - Read metadata for a given cell
2. **find_cells_with_stress_temperature** - Find cells by stress temperature
3. **find_cells_with_stress_wavelength** - Find cells by stress wavelength
4. **find_cells_with_pixel_number** - Find cells with a specific pixel number
5. **plot_iv_data** - Plot IV data for a cell/pixel/date
6. **list_data_for_cell** - List all measurements for a cell
7. **list_data_for_cell_date_match** - List measurements where Current, IV, and Power have matching dates
8. **get_power_data** - Retrieve Power data
9. **get_current_data** - Retrieve Current data
10. **get_iv_data** - Retrieve IV data
11. **retrieve_and_store_data** - Batch retrieve and store data
12. **get_data_from_hdf5** - Retrieve data for a given group path
13. **calculate_eqe** - Calculate EQE curve
14. **get_all_dates** - Get all available dates for a cell/pixel
15. **plot_eqe_over_time** - Plot EQE curves over time
16. **calculate_jsc** - Calculate short circuit current density
17. **calculate_voc** - Calculate open circuit voltage
18. **calculate_v_pmax** - Calculate voltage at max power point
19. **calculate_j_pmax** - Calculate current density at max power point
20. **calculate_fill_factor** - Calculate fill factor
21. **calculate_pce** - Calculate power conversion efficiency
22. **convert_iv_to_jv** - Convert IV to JV data

In [ ]:
# ============================================================================
# IMPORT LIBRARIES
# ============================================================================
import h5py              # For reading HDF5 files
import matplotlib.pyplot as plt  # For creating plots and visualizations
import pandas as pd      # For working with tabular data (DataFrames)
import numpy as np       # For numerical operations and calculations

# ============================================================================
# CONFIGURATION
# ============================================================================
# UPDATED FOR FA25: Changed default file path
hdf5_path = 'data_fa25.h5'  # Path to the FA25 HDF5 database file

# Physical constants used for EQE calculations
h = 6.62607015e-34   # Planck's constant in J·s (joule-seconds)
c = 2.998e8          # Speed of light in m/s (meters per second)
e = 1.602176634e-19  # Elementary charge in C (coulombs)

# ============================================================================
# METADATA QUERY FUNCTIONS
# ============================================================================

def read_metadata(hdf5_path, cell_number):
    """
    Read and display metadata for a given cell from the HDF5 file.

    UPDATED FOR FA25: Metadata is now stored at the CELL level, not pixel level.
    FA25 cells use 3-digit numbers (125-197).

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file containing the solar cell data.
    cell_number : str or int
        Cell number (e.g., 129, 156, or '129'). Will be automatically zero-padded to 3 digits.

    Returns:
    --------
    None
        Prints metadata to console.

    Example:
    --------
    read_metadata(hdf5_path, 129)
    # Output: Stress Temperature: 55, Stress Wavelength: White
    """
    # FA25: Ensure 3-digit zero-padding for cell numbers (e.g., 129 -> '129')
    cell_number = str(cell_number).zfill(3)

    # Open the HDF5 file in read mode using context manager (automatically closes file)
    with h5py.File(hdf5_path, 'r') as hdf:
        # Try to get the group for the specified cell
        cell_group = hdf.get(f'Cell{cell_number}')

        # Check if cell exists in database
        if cell_group is None:
            print(f"Cell{cell_number} not found in the HDF5 file.")
            return

        # Display metadata stored at the cell level
        print(f"Metadata for Cell{cell_number}:")
        # FA25: Metadata is at CELL level (not pixel level like in SP25)
        if cell_group.attrs:
            print("  Cell-level metadata:")
            # Loop through all metadata attributes and display them
            for key, value in cell_group.attrs.items():
                print(f"    {key}: {value}")
        else:
            print("  No metadata found at cell level.")


def find_cells_with_stress_temperature(hdf5_path, target_temperature):
    """
    Find all cells that have a specific stress temperature in their metadata.

    Useful for filtering cells based on experimental conditions.
    UPDATED FOR FA25: Metadata is now at CELL level, not pixel level.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    target_temperature : float or str
        Target stress temperature to search for in degrees Celsius (e.g., 55).

    Returns:
    --------
    list
        List of cell numbers (as strings like 'Cell129') that match the target temperature.

    Example:
    --------
    cells_at_55C = find_cells_with_stress_temperature(hdf5_path, 55)
    # Returns: ['Cell125', 'Cell126', 'Cell127', ...]
    """
    cells_with_target_temp = []  # Initialize empty list to store matching cells

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Iterate over each cell group in the HDF5 file
        for cell in hdf.keys():
            cell_group = hdf[cell]

            # FA25: Check at CELL level (not pixel level)
            if 'Stress Temperature' in cell_group.attrs:
                # Try to compare temperatures (handle potential conversion errors)
                try:
                    stress_temp = float(cell_group.attrs['Stress Temperature'])
                    # If temperatures match, add this cell to our results list
                    if stress_temp == float(target_temperature):
                        cells_with_target_temp.append(cell)
                except ValueError:
                    # If we can't convert to float, print warning and skip this cell
                    print(f"Warning: Could not convert Stress Temperature to float for cell {cell}")

    return cells_with_target_temp


def find_cells_with_stress_wavelength(hdf5_path, target_wavelength):
    """
    Find all cells that have a specific stress wavelength in their metadata.

    Useful for filtering cells based on light stress conditions.
    UPDATED FOR FA25: Metadata is now at CELL level, not pixel level.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    target_wavelength : str
        Target stress wavelength to search for (e.g., 'White' for white light,
        or specific wavelength like '470' for blue light).

    Returns:
    --------
    list
        List of cell numbers that match the target wavelength.

    Example:
    --------
    white_light_cells = find_cells_with_stress_wavelength(hdf5_path, 'White')
    """
    cells_with_target_wavelength = []

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Iterate over each cell in the HDF5 file
        for cell in hdf.keys():
            cell_group = hdf[cell]

            # FA25: Check at CELL level (not pixel level)
            if 'Stress Wavelength' in cell_group.attrs:
                # Case-insensitive comparison of wavelength strings
                if str(cell_group.attrs['Stress Wavelength']).lower() == str(target_wavelength).lower():
                    cells_with_target_wavelength.append(cell)

    return cells_with_target_wavelength


def find_cells_with_pixel_number(hdf5_path, target_pixel_number):
    """
    Find all cells that have data for a specific pixel number.

    Useful when you want to analyze only cells where a particular pixel was measured.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    target_pixel_number : int
        Target pixel number to search for (1-8 for FA25).

    Returns:
    --------
    list
        List of cell numbers that have the target pixel number.

    Example:
    --------
    cells_with_pixel_5 = find_cells_with_pixel_number(hdf5_path, 5)
    """
    cells_with_target_pixel = []

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Iterate over each cell in the HDF5 file
        for cell in hdf.keys():
            cell_group = hdf[cell]

            # Iterate over each subgroup in the cell (could be Power, Pixel1, Pixel2, etc.)
            for pixel in cell_group.keys():
                # Check if this subgroup matches our target pixel
                if pixel == f'Pixel{target_pixel_number}':
                    cells_with_target_pixel.append(cell)
                    break  # Found the pixel, no need to check other pixels in this cell

    return cells_with_target_pixel


# ============================================================================
# DATA LISTING FUNCTIONS
# ============================================================================

def list_data_for_cell(hdf5_path, cell_number):
    """
    List all measurement data available for a given cell.

    Shows all dates for which Power, Current, and IV measurements exist.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell_number : int or str
        Cell number (e.g., 129).

    Returns:
    --------
    list
        List of datasets for the specified cell (currently returns empty list,
        but prints all available data to console).

    Example:
    --------
    list_data_for_cell(hdf5_path, 129)
    # Prints all measurement dates and types for Cell 129
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell_number).zfill(3)
    dataset_list = []

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Get the group for the specified cell
        cell_group = hdf.get(f'Cell{cell_number}')
        if cell_group is None:
            print(f"Cell{cell_number} not found in the HDF5 file.")
            return dataset_list

        print(f"Listing all measurement data for Cell{cell_number}:")

        # Iterate over each subgroup in the cell (Power, Pixel1, Pixel2, etc.)
        for pixel in cell_group.keys():
            if pixel == 'Power':
                # Power data is stored at cell level (one measurement per cell)
                print(f"Power:")
                power_group = cell_group[pixel]
                # List all date-based datasets in the Power group
                for dataset in power_group.keys():
                    print(f"  Dataset: {dataset}")
            else:
                # Pixel-specific data (Current and IV measurements)
                print(f"Pixel: {pixel}")
                pixel_group = cell_group[pixel]
                # List all measurement types (Current, IV) for this pixel
                for group in pixel_group.keys():
                    print(f"  Group: {group}")
                    data_group = pixel_group[group]
                    # List all dates for this measurement type
                    for dataset in data_group.keys():
                        print(f"    Dataset: {dataset}")

    return dataset_list


def list_data_for_cell_date_match(hdf5_path, cell_number):
    """
    List measurement data where Current, IV, and Power all have the same date.

    This is useful for finding complete measurement sets where all three types
    of data were collected on the same day.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell_number : int or str
        Cell number (e.g., 129).

    Returns:
    --------
    list
        List of datasets for the specified cell with matching dates.

    Example:
    --------
    list_data_for_cell_date_match(hdf5_path, 129)
    # Shows only dates where Power, Current, AND IV data all exist
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell_number).zfill(3)
    dataset_list = []

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Get the group for the specified cell
        cell_group = hdf.get(f'Cell{cell_number}')
        if cell_group is None:
            print(f"Cell{cell_number} not found in the HDF5 file.")
            return dataset_list

        print(f"Listing measurement data that have common dates for Cell{cell_number}:")

        # Initialize sets to store dates for each measurement type
        current_dates = set()
        iv_dates = set()
        power_dates = set()

        # Collect all dates for each measurement type
        for pixel in cell_group.keys():
            if pixel == 'Power':
                power_group = cell_group[pixel]
                power_dates = set(power_group.keys())
            else:
                pixel_group = cell_group[pixel]
                # Add dates from Current measurements
                if 'Current' in pixel_group:
                    current_group = pixel_group['Current']
                    current_dates.update(current_group.keys())
                # Add dates from IV measurements
                if 'IV' in pixel_group:
                    iv_group = pixel_group['IV']
                    iv_dates.update(iv_group.keys())

        # Find dates that appear in ALL three measurement types (intersection)
        common_dates = current_dates & iv_dates & power_dates

        # Display results for each common date
        for date in sorted(common_dates):
            print(f"Common Date: {date}")
            # Show which pixels have complete data for this date
            for pixel in cell_group.keys():
                if pixel != 'Power':
                    pixel_group = cell_group[pixel]
                    # Check if this pixel has both Current and IV data for this date
                    if ('Current' in pixel_group and date in pixel_group['Current'] and
                        'IV' in pixel_group and date in pixel_group['IV']):
                        print(f"  {pixel}")

    return dataset_list


# ============================================================================
# DATA RETRIEVAL FUNCTIONS
# ============================================================================

def get_power_data(hdf5_path, cell_number, date):
    """
    Retrieve Power data (wavelength vs power) for a given cell and date.

    Power is measured once per cell (not per pixel).
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell_number : str or int
        Cell number (e.g., 129 or '129').
    date : str
        Date in the format 'YYYY_MM_DD' (e.g., '2025_09_23').

    Returns:
    --------
    pd.DataFrame or None
        DataFrame containing columns ['Wavelength (nm)', 'Power (W)'],
        or None if not found.

    Example:
    --------
    power_df = get_power_data(hdf5_path, 129, '2025_09_23')
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell_number).zfill(3)

    with h5py.File(hdf5_path, 'r') as hdf:
        # Construct the full path to the Power data for this cell and date
        group_path = f'Cell{cell_number}/Power/{date}'

        # Check if the data exists at this path
        if group_path in hdf:
            group = hdf[group_path]
            # Read the numerical data array
            data = group['Data'][:]
            # Read and decode the column headers (stored as bytes)
            headers = [h.decode('utf-8') for h in group['Headers'][:]]
            # Create a pandas DataFrame with the data and headers
            df = pd.DataFrame(data, columns=headers)
            return df
        else:
            print(f"Power data for Cell{cell_number} on {date} not found.")
            return None


def get_current_data(hdf5_path, cell_number, pixel_number, date):
    """
    Retrieve Current data (wavelength vs current) for a specific cell, pixel, and date.

    Current is measured separately for each pixel.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell_number : str or int
        Cell number (e.g., 129 or '129').
    pixel_number : int
        Pixel number (1-8 for FA25).
    date : str
        Date in the format 'YYYY_MM_DD'.

    Returns:
    --------
    pd.DataFrame or None
        DataFrame containing columns ['Wavelength (nm)', 'Current (A)'],
        or None if not found.

    Example:
    --------
    current_df = get_current_data(hdf5_path, 129, 4, '2025_09_23')
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell_number).zfill(3)
    pixel_number = str(pixel_number)

    with h5py.File(hdf5_path, 'r') as hdf:
        # Construct the full path to the Current data
        group_path = f'Cell{cell_number}/Pixel{pixel_number}/Current/{date}'

        if group_path in hdf:
            group = hdf[group_path]
            data = group['Data'][:]
            headers = [h.decode('utf-8') for h in group['Headers'][:]]
            df = pd.DataFrame(data, columns=headers)
            return df
        else:
            print(f"Current data for Cell{cell_number}, Pixel{pixel_number} on {date} not found.")
            return None


def get_iv_data(hdf5_path, cell_number, pixel_number, date):
    """
    Retrieve IV data (voltage vs current) for a specific cell, pixel, and date.

    IV data includes both forward and reverse scan measurements.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell_number : str or int
        Cell number (e.g., 129 or '129').
    pixel_number : int
        Pixel number (1-8 for FA25).
    date : str
        Date in the format 'YYYY_MM_DD'.

    Returns:
    --------
    pd.DataFrame or None
        DataFrame containing columns ['Voltage (V)', 'Forward Scan (mA)', 'Reverse Scan (mA)'],
        or None if not found.

    Example:
    --------
    iv_df = get_iv_data(hdf5_path, 129, 4, '2025_09_23')
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell_number).zfill(3)
    pixel_number = str(pixel_number)

    with h5py.File(hdf5_path, 'r') as hdf:
        # Construct the full path to the IV data
        group_path = f'Cell{cell_number}/Pixel{pixel_number}/IV/{date}'

        if group_path in hdf:
            group = hdf[group_path]
            data = group['Data'][:]
            headers = [h.decode('utf-8') for h in group['Headers'][:]]
            df = pd.DataFrame(data, columns=headers)
            return df
        else:
            print(f"IV data for Cell{cell_number}, Pixel{pixel_number} on {date} not found.")
            return None


def retrieve_and_store_data(hdf5_path, cell_pixel_date_info):
    """
    Batch retrieve and store data from the HDF5 file as global variables.

    This function creates global variables with names like:
    - cell129_power_2025_09_23
    - cell129_pixel4_current_2025_09_23
    - cell129_pixel4_iv_2025_09_23

    UPDATED FOR FA25: Now handles 3-digit cell numbers.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell_pixel_date_info : dict
        Dictionary with structure:
        {
            cell_number: {
                'pixels': [list of pixel numbers],
                'dates': [list of dates as 'YYYY_MM_DD']
            }
        }

    Example:
    --------
    info = {
        129: {
            'pixels': [1, 4],
            'dates': ['2025_09_23']
        }
    }
    retrieve_and_store_data(hdf5_path, info)
    # Creates: cell129_power_2025_09_23, cell129_pixel1_current_2025_09_23, etc.
    """
    # Loop through each cell in the input dictionary
    for cell, info in cell_pixel_date_info.items():
        # Extract the list of pixels and dates for this cell
        pixels = info['pixels']
        dates = info['dates']

        # Loop through each pixel
        for pixel in pixels:
            # Loop through each date
            for date in dates:
                # Retrieve Power data (one per cell, not per pixel)
                power_data = get_power_data(hdf5_path, cell, date)
                if power_data is not None:
                    # Create a descriptive variable name
                    var_name = f'cell{cell}_power_{date}'
                    # Store the DataFrame as a global variable
                    globals()[var_name] = power_data

                # Retrieve Current data (specific to this pixel)
                current_data = get_current_data(hdf5_path, cell, pixel, date)
                if current_data is not None:
                    var_name = f'cell{cell}_pixel{pixel}_current_{date}'
                    globals()[var_name] = current_data

                # Retrieve IV data (specific to this pixel)
                iv_data = get_iv_data(hdf5_path, cell, pixel, date)
                if iv_data is not None:
                    var_name = f'cell{cell}_pixel{pixel}_iv_{date}'
                    globals()[var_name] = iv_data


def get_data_from_hdf5(hdf5_path, group_path):
    """
    Generic function to retrieve data from any HDF5 group path.

    Lower-level function used by other data retrieval functions.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    group_path : str
        Full path to the data group within the HDF5 file
        (e.g., 'Cell129/Pixel4/Current/2025_09_23').

    Returns:
    --------
    pd.DataFrame or None
        DataFrame containing the data and headers, or None if path not found.
    """
    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Check if the specified path exists
        if group_path in hdf:
            # Access the group at this path
            group = hdf[group_path]

            # Read the data array and headers
            data = group['Data'][:]
            headers = [h.decode('utf-8') for h in group['Headers'][:]]

            # Create and return a DataFrame
            df = pd.DataFrame(data, columns=headers)
            return df
        else:
            # Path not found, print error message
            print(f"Data for {group_path} not found.")
            return None


# ============================================================================
# EQE CALCULATION FUNCTIONS
# ============================================================================

def calculate_eqe(hdf5_path, cell, pixel, date):
    """
    Calculate the External Quantum Efficiency (EQE) curve for a given cell/pixel/date.

    EQE represents the ratio of electrons collected to photons incident on the cell.
    Formula: EQE(λ) = (I(λ) * h * c) / (P(λ) * λ * e) * 100%

    Where:
    - I(λ) = measured current at wavelength λ
    - P(λ) = incident power at wavelength λ
    - h = Planck's constant
    - c = speed of light
    - e = elementary charge

    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell : int or str
        Cell number (e.g., 129).
    pixel : int
        Pixel number (1-8).
    date : str
        Date in the format 'YYYY_MM_DD'.

    Returns:
    --------
    pd.DataFrame or None
        DataFrame with columns ['Wavelength (nm)', 'EQE (%)'],
        or None if data not found or wavelengths don't match.

    Example:
    --------
    eqe_df = calculate_eqe(hdf5_path, 129, 4, '2025_09_23')
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell).zfill(3)
    pixel_number = str(pixel)

    # Construct the paths to Power and Current data
    power_group_path = f'Cell{cell_number}/Power/{date}'
    current_group_path = f'Cell{cell_number}/Pixel{pixel_number}/Current/{date}'

    # Retrieve both datasets
    power_data = get_data_from_hdf5(hdf5_path, power_group_path)
    current_data = get_data_from_hdf5(hdf5_path, current_group_path)

    # Check if both datasets were successfully retrieved
    if power_data is None or current_data is None:
        return None

    # Verify that wavelengths match between power and current measurements
    if not power_data['Wavelength (nm)'].equals(current_data['Wavelength (nm)']):
        print("Wavelengths do not match between Power and Current datasets.")
        return None

    # Extract the wavelength, power, and current arrays
    wavelength = power_data['Wavelength (nm)']  # in nanometers
    power = power_data['Power (W)']             # in watts
    current = current_data['Current (A)']       # in amperes

    # Convert wavelength from nanometers to meters for calculation
    wavelength_m = wavelength * 1e-9

    # Calculate EQE using the formula: EQE = (I * h * c) / (P * λ * e) * 100%
    # This gives the percentage of photons converted to electrons
    eqe = (current * h * c) / (power * wavelength_m * e) * 100

    # Create a DataFrame with the results
    eqe_data = pd.DataFrame({
        'Wavelength (nm)': wavelength,
        'EQE (%)': eqe
    })

    return eqe_data


def get_all_dates(hdf5_path, cell, pixel):
    """
    Retrieve all available measurement dates for a given cell and pixel.

    Returns the union of dates from both Power and Current measurements.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell : int or str
        Cell number (e.g., 129).
    pixel : int
        Pixel number (1-8).

    Returns:
    --------
    list
        Sorted list of available dates as strings in 'YYYY_MM_DD' format.

    Example:
    --------
    dates = get_all_dates(hdf5_path, 129, 4)
    # Returns: ['2025_09_08', '2025_09_15', '2025_09_23', ...]
    """
    # FA25: Ensure 3-digit zero-padding
    cell_number = str(cell).zfill(3)
    pixel_number = str(pixel)
    dates = set()  # Use a set to avoid duplicates

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Construct paths to Power and Current groups
        power_group_path = f'Cell{cell_number}/Power'
        current_group_path = f'Cell{cell_number}/Pixel{pixel_number}/Current'

        # Add all Power measurement dates to the set
        if power_group_path in hdf:
            dates.update(hdf[power_group_path].keys())

        # Add all Current measurement dates to the set
        if current_group_path in hdf:
            dates.update(hdf[current_group_path].keys())

    # Return sorted list of dates
    return sorted(dates)


# ============================================================================
# PLOTTING FUNCTIONS
# ============================================================================

def plot_iv_data(hdf5_path, cell, pixel, date):
    """
    Plot IV curves (current vs voltage) for a specific cell, pixel, and date.

    Creates a plot showing both forward and reverse voltage scans.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell : str or int
        Cell number (e.g., 129 or '129').
    pixel : int
        Pixel number (1-8).
    date : str
        Date in the format 'YYYY_MM_DD'.

    Returns:
    --------
    None
        Displays a matplotlib plot.

    Example:
    --------
    plot_iv_data(hdf5_path, 129, 4, '2025_09_23')
    """
    # FA25: Ensure 3-digit zero-padding
    cell = str(cell).zfill(3)

    # Open the HDF5 file in read mode
    with h5py.File(hdf5_path, 'r') as hdf:
        # Construct the path to IV data
        group_path = f'Cell{cell}/Pixel{pixel}/IV/{date}'

        if group_path in hdf:
            group = hdf[group_path]
            # Read data and headers
            data = group['Data'][:]
            headers = [h.decode('utf-8') for h in group['Headers'][:]]

            # Extract voltage and current data from the array
            voltage = data[:, 0]       # First column: voltage
            forward_scan = data[:, 1]  # Second column: forward scan current
            reverse_scan = data[:, 2]  # Third column: reverse scan current

            # Create the plot
            plt.figure(figsize=(10, 6))
            plt.plot(voltage, forward_scan, label='Forward Scan (mA)')
            plt.plot(voltage, reverse_scan, label='Reverse Scan (mA)')
            plt.xlabel(headers[0])  # Use actual header for x-axis label
            plt.ylabel('Current (mA)')
            plt.title(f'IV Data for Cell {cell}, Pixel {pixel} on {date}')
            plt.legend()
            plt.grid(True)
            plt.show()
        else:
            print(f"No IV data found for Cell {cell}, Pixel {pixel} on {date}")


def plot_eqe_over_time(hdf5_path, cell, pixel):
    """
    Plot EQE curves over time for a given cell and pixel.

    Creates a single plot with multiple EQE curves, one for each measurement date.
    Useful for visualizing how EQE degrades over the stress period.
    UPDATED FOR FA25: Cell numbers now use 3-digit padding.

    Parameters:
    -----------
    hdf5_path : str
        Path to the HDF5 file.
    cell : int or str
        Cell number (e.g., 129).
    pixel : int
        Pixel number (1-8).

    Returns:
    --------
    None
        Displays a matplotlib plot with multiple curves.

    Example:
    --------
    plot_eqe_over_time(hdf5_path, 129, 4)
    # Shows EQE degradation over all measurement dates
    """
    # Get all available measurement dates for this cell/pixel
    dates = get_all_dates(hdf5_path, cell, pixel)

    # Check if any dates were found
    if not dates:
        print(f"No dates found for Cell {cell}, Pixel {pixel}.")
        return

    # Create a new figure
    plt.figure(figsize=(10, 6))

    # Calculate and plot EQE for each date
    for date in dates:
        eqe_data = calculate_eqe(hdf5_path, cell, pixel, date)
        if eqe_data is not None:
            # Plot this EQE curve with date in the legend
            plt.plot(eqe_data['Wavelength (nm)'], eqe_data['EQE (%)'], label=f'Date: {date}')

    # Set plot labels and styling
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('EQE (%)')
    plt.title(f'EQE Curves for Cell {cell}, Pixel {pixel} Over Time')
    plt.legend()
    plt.grid(True)
    plt.show()


# ============================================================================
# SOLAR CELL PERFORMANCE CALCULATION FUNCTIONS
# ============================================================================

def calculate_jsc(jv_data, scan_type):
    """
    Calculate the Short Circuit Current Density (Jsc) from JV data.

    Jsc is the current density when voltage = 0V. It represents the maximum
    current the solar cell can produce.

    Parameters:
    -----------
    jv_data : pd.DataFrame
        DataFrame containing JV data with columns 'Voltage (V)' and current density columns.
    scan_type : str
        Column name for the scan type to use:
        - 'Forward Scan (mA/cm^2)' or
        - 'Reverse Scan (mA/cm^2)'

    Returns:
    --------
    float
        Short Circuit Current Density (Jsc) in mA/cm^2.

    Example:
    --------
    jsc = calculate_jsc(jv_data, 'Reverse Scan (mA/cm^2)')
    """
    # Find the row index where Voltage is exactly zero
    jsc_index = jv_data.index[jv_data['Voltage (V)'] == 0]

    # Get the current density at that voltage and return absolute value
    # (solar cells produce negative current, but we report positive values)
    jsc = abs(jv_data.loc[jsc_index, scan_type].values[0])
    return jsc


def calculate_voc(jv_data, scan_type):
    """
    Calculate the Open Circuit Voltage (Voc) from JV data.

    Voc is the voltage when current = 0A. It represents the maximum voltage
    the solar cell can produce. Uses interpolation to find the exact voltage
    where current crosses zero.

    Parameters:
    -----------
    jv_data : pd.DataFrame
        DataFrame containing the JV data.
    scan_type : str
        Column name for the scan type ('Forward Scan (mA/cm^2)' or 'Reverse Scan (mA/cm^2)').

    Returns:
    --------
    float
        Open Circuit Voltage (Voc) in V.

    Example:
    --------
    voc = calculate_voc(jv_data, 'Reverse Scan (mA/cm^2)')
    """
    # Interpolate to find the voltage at zero current
    # (current may not be exactly zero at any measured point)
    voc = np.interp(0, jv_data[scan_type], jv_data['Voltage (V)'])
    return voc


def calculate_v_pmax(jv_data, scan_type):
    """
    Calculate the voltage at the maximum power point (V_pmax) from JV data.

    The maximum power point is where the product of voltage and current is largest.
    This is the optimal operating point for the solar cell.

    Parameters:
    -----------
    jv_data : pd.DataFrame
        DataFrame containing the JV data.
    scan_type : str
        Column name for the scan type ('Forward Scan (mA/cm^2)' or 'Reverse Scan (mA/cm^2)').

    Returns:
    --------
    float
        Voltage at maximum power point (V_pmax) in V.

    Example:
    --------
    v_pmax = calculate_v_pmax(jv_data, 'Reverse Scan (mA/cm^2)')
    """
    # Calculate power at each measured point (P = V × I)
    jv_data['Power (mW/cm^2)'] = jv_data['Voltage (V)'] * jv_data[scan_type]

    # Find the index where power is at minimum (most negative = maximum absolute power)
    pmax_index = np.argmin(jv_data['Power (mW/cm^2)'])

    # Get the voltage at that point
    v_pmax = jv_data.loc[pmax_index, 'Voltage (V)']
    return v_pmax


def calculate_j_pmax(jv_data, scan_type):
    """
    Calculate the current density at the maximum power point (J_pmax) from JV data.

    This is the current density at the point where power output is maximized.

    Parameters:
    -----------
    jv_data : pd.DataFrame
        DataFrame containing the JV data.
    scan_type : str
        Column name for the scan type ('Forward Scan (mA/cm^2)' or 'Reverse Scan (mA/cm^2)').

    Returns:
    --------
    float
        Current density at maximum power point (J_pmax) in mA/cm^2.

    Example:
    --------
    j_pmax = calculate_j_pmax(jv_data, 'Reverse Scan (mA/cm^2)')
    """
    # Calculate power at each point
    jv_data['Power (mW/cm^2)'] = jv_data['Voltage (V)'] * jv_data[scan_type]

    # Find the index where power is at maximum
    pmax_index = np.argmin(jv_data['Power (mW/cm^2)'])

    # Get the current density at that point and return absolute value
    # (solar cells produce negative current, but we report positive values)
    j_pmax = abs(jv_data.loc[pmax_index, scan_type])
    return j_pmax


def calculate_fill_factor(jsc, voc, v_pmax, j_pmax):
    """
    Calculate the Fill Factor (FF) from solar cell parameters.

    Fill Factor represents how 'square' the IV curve is. It's the ratio of
    the maximum power to the theoretical maximum (Jsc × Voc).

    Formula: FF = (V_pmax × J_pmax) / (Voc × Jsc)

    Higher FF (closer to 1.0) indicates better solar cell quality.

    Parameters:
    -----------
    jsc : float
        Short Circuit Current Density (Jsc) in mA/cm^2.
    voc : float
        Open Circuit Voltage (Voc) in V.
    v_pmax : float
        Voltage at maximum power point (V_pmax) in V.
    j_pmax : float
        Current density at maximum power point (J_pmax) in mA/cm^2.

    Returns:
    --------
    float
        Fill Factor (FF), dimensionless value typically between 0 and 1.

    Example:
    --------
    ff = calculate_fill_factor(jsc, voc, v_pmax, j_pmax)
    """
    # Calculate actual maximum power
    max_power = v_pmax * j_pmax

    # Fill Factor is the ratio of actual max power to theoretical max power
    ff = max_power / (jsc * voc)
    return ff


def calculate_pce(jsc, voc, ff):
    """
    Calculate the Power Conversion Efficiency (PCE) from solar cell parameters.

    PCE represents the percentage of incident light power converted to electrical power.

    Formula: PCE = (Jsc × Voc × FF) / P_incident × 100%

    Where P_incident is the incident power from the solar simulator (99.8 mW/cm^2).

    Parameters:
    -----------
    jsc : float
        Short Circuit Current Density (Jsc) in mA/cm^2.
    voc : float
        Open Circuit Voltage (Voc) in V.
    ff : float
        Fill Factor (FF), dimensionless.

    Returns:
    --------
    float
        Power Conversion Efficiency (PCE) as a percentage.

    Example:
    --------
    pce = calculate_pce(jsc, voc, ff)
    """
    # Incident power from the solar simulator in mW/cm^2
    incident_power = 99.8  # mW/cm^2

    # Calculate maximum power output
    max_power_output = jsc * voc * ff

    # PCE is the ratio of output power to incident power, as a percentage
    # Negative sign converts from negative current convention to positive efficiency
    pce = -(max_power_output / incident_power) * 100  # %
    return pce


def convert_iv_to_jv(iv_data, area = 0.14):
    """
    Convert IV data (current in mA) to JV data (current density in mA/cm^2).

    Current density is calculated by dividing current by the active area of the pixel.
    This normalization allows comparison between pixels of different sizes.

    NOTE: FA25 uses unmasked pixel area of 0.14 cm^2 (default).

    Parameters:
    -----------
    iv_data : pd.DataFrame
        DataFrame containing IV data with columns:
        - 'Voltage (V)'
        - 'Forward Scan (mA)'
        - 'Reverse Scan (mA)'
    area : float, optional
        Active area of the solar cell pixel in cm^2. Default is 0.14 cm^2 for FA25.

    Returns:
    --------
    pd.DataFrame
        DataFrame containing JV data with columns:
        - 'Voltage (V)'
        - 'Forward Scan (mA/cm^2)'
        - 'Reverse Scan (mA/cm^2)'

    Example:
    --------
    jv_data = convert_iv_to_jv(iv_data, area=0.14)
    """
    # Calculate current density by dividing current by area
    iv_data['Forward Scan (mA/cm^2)'] = iv_data['Forward Scan (mA)'] / area
    iv_data['Reverse Scan (mA/cm^2)'] = iv_data['Reverse Scan (mA)'] / area

    # Remove the original current columns (keep only current density)
    jv_data = iv_data.drop(columns=['Forward Scan (mA)', 'Reverse Scan (mA)'])

    return jv_data

# Query Cell Metadata

In [ ]:
# Example usage - FA25 cells use 3-digit numbers (125-197)
cell_number = 129  # Can also use '129' as a string
read_metadata(hdf5_path, cell_number)

# Query Temperature Metadata

In [ ]:
# Example usage - FA25: All cells stressed at 55°C
target_temperature = 55
cells = find_cells_with_stress_temperature(hdf5_path, target_temperature)
print(f"There are {len(cells)} cells with stress temperature of {target_temperature}°C: {cells}")

# Query Wavelength Metadata

In [ ]:
# Example usage - FA25: All cells stressed with white light
target_wavelength = 'White'  # Note: Stored as 'White' not 'White light'
cells = find_cells_with_stress_wavelength(hdf5_path, target_wavelength)
print(f"There are {len(cells)} cells with stress wavelength of '{target_wavelength}': {cells}")

# Query Pixel Number

In [ ]:
# Example usage - FA25: Find all cells with pixel 6 data
target_pixel_number = 6
cells = find_cells_with_pixel_number(hdf5_path, target_pixel_number)
print(f"There are {len(cells)} cells with pixel number {target_pixel_number}: {cells}")

# Query All Measurement Data for a Cell
This will return a list all measurements for a given cell number that exist in the HDF5 file.

In [ ]:
cell_number = 129 # Replace with desired FA25 cell number (125-197)
dataset_list = list_data_for_cell(hdf5_path, cell_number)
for item in dataset_list:
    print(item)

# Query Measurement Data with a Common Date for a Cell
This will return a list of measurements for a given cell number that have matching dates, grouped by date. If a pixel is listed under that date, then there exists a complete set of measurements (current, power, and IV) for that pixel on that date.

In [ ]:
cell_number = 129  # Replace with desired FA25 cell number (125-197)
dataset_list = list_data_for_cell_date_match(hdf5_path, cell_number)
for item in dataset_list:
    print(item)

# Getting Data from a single Cell/Pixel/Date

In [ ]:
cell_number = 129 # Replace with desired FA25 cell number (125-197)
pixel_number = 4 # Replace with the desired pixel number (1-8)
date = '2025_09_23' # Replace with the desired date formatted as 'YYYY_MM_DD'

power_data = get_power_data(hdf5_path, cell_number, date)
if power_data is not None:
    print("Power Data:")
    print(power_data)

current_data = get_current_data(hdf5_path, cell_number, pixel_number, date)
if current_data is not None:
    print("Current Data:")
    print(current_data)

iv_data = get_iv_data(hdf5_path, cell_number, pixel_number, date)
if iv_data is not None:
    print("IV Data:")
    print(iv_data)

# Getting Data from Multiple Cells/Pixels/Dates

In [ ]:
# Define the dictionary with the cells, pixels, and dates to retrieve from the HDF5 file
# FA25: Use 3-digit cell numbers (125-197)
cell_pixel_date_info = {
    129: { # Cell number
        'pixels': [1, 4], # Pixel numbers
        'dates': ['2025_09_23'] # Dates
    },
    136: {
        'pixels': [1],
        'dates': ['2025_09_23']
    },
    144: {
        'pixels': [2],
        'dates': ['2025_09_23']
    }
}

# Call the function to retrieve and store the data from the HDF5 file based on the dictionary
# defined above. This will create global DataFrames for each measurement data with names based on
# the cell, pixel, and date. For example, the Power data for Cell 129 on 2025_09_23 will be
# stored in a global DataFrame named cell129_power_2025_09_23 and the Current data for Cell 129, Pixel 1
# on the same date will be stored in a global DataFrame named cell129_pixel1_current_2025_09_23. Similarly,
# the IV data for Cell 129, Pixel 1 on the same date will be stored in a global DataFrame named
# cell129_pixel1_iv_2025_09_23.

retrieve_and_store_data(hdf5_path, cell_pixel_date_info)

# Example usage: Print the contents of the global DataFrames. Uncomment the lines below to run.
print(cell129_power_2025_09_23)
print(cell136_pixel1_current_2025_09_23)
print(cell144_pixel2_iv_2025_09_23)

# Calculate EQE for a given cell/pixel/date

In [ ]:
cell = 129  # FA25 cell number
pixel = 1
date = '2025_09_23'

eqe_data = calculate_eqe(hdf5_path, cell, pixel, date)
if eqe_data is not None:
    print("EQE Data:")
    print(eqe_data)

# Plot EQE over time

In [ ]:
cell = 129  # FA25 cell number
pixel = 4
plot_eqe_over_time(hdf5_path, cell, pixel)

# Plot IV Data

In [ ]:
# Plot IV data for a FA25 cell
cell = 129 # Replace with desired FA25 cell number (125-197)
pixel = 2 # Replace with the desired pixel number (1-8)
date = '2025_09_23'  # Replace with the desired date
plot_iv_data(hdf5_path, cell, pixel, date)

# Calculate PCE and Related Quantities

In [ ]:
# Define the dictionary with the cells, pixels, and dates to retrieve from the HDF5 file
# FA25: Use 3-digit cell numbers (125-197)
cell_pixel_date_info = {
    129: { # Cell number
        'pixels': [1, 2], # Pixel numbers
        'dates': ['2025_09_23', '2025_09_30'] # Dates
    },
    136: {
        'pixels': [1, 2],
        'dates': ['2025_09_23']
    },
    144: {
        'pixels': [1],
        'dates': ['2025_09_23']
    }
}

# Choose between 'Forward Scan (mA/cm^2)' and 'Reverse Scan (mA/cm^2)'. Uncomment the desired option.
scan_type = 'Reverse Scan (mA/cm^2)'
#scan_type = 'Forward Scan (mA/cm^2)'

# Iterate over each cell in the dictionary
for cell_number, info in cell_pixel_date_info.items():
    # Iterate over each pixel in the current cell
    for pixel_number in info['pixels']:
        # FA25: Unmasked area is 0.14 cm^2
        area = 0.14
        if area is None:
            # If the area is not found, skip to the next pixel
            continue
        # Iterate over each date for the current cell and pixel
        for date in info['dates']:
            # Retrieve IV data for the specified cell, pixel, and date
            iv_data = get_iv_data(hdf5_path, cell_number, pixel_number, date)
            if iv_data is not None:
                # Convert IV data to JV data using the area
                jv_data = convert_iv_to_jv(iv_data, area)
                # Calculate Jsc, Voc, V_pmax, J_pmax, FF, and PCE
                jsc = calculate_jsc(jv_data, scan_type)
                voc = calculate_voc(jv_data, scan_type)
                v_pmax = calculate_v_pmax(jv_data, scan_type)
                j_pmax = calculate_j_pmax(jv_data, scan_type)
                ff = calculate_fill_factor(jsc, voc, v_pmax, j_pmax)
                pce = np.abs(calculate_pce(jsc, voc, ff))

                # Print the results
                print(f"Cell {cell_number}, Pixel {pixel_number}, Date {date}:")
                print(f"Short Circuit Current Density (Jsc): {abs(jsc):.2f} mA/cm^2")
                print(f"Open Circuit Voltage (Voc): {voc} V")
                print(f"Voltage at Maximum Power Point (V_pmax): {v_pmax} V")
                print(f"Current Density at Maximum Power Point (J_pmax): {abs(j_pmax):.2f} mA/cm^2")
                print(f"Fill Factor (FF): {ff}")
                print(f"Power Conversion Efficiency (PCE): {pce} %")
                print()

# Introduction to HDF5 Files

## What is HDF5?

HDF5 (Hierarchical Data Format version 5) is a file format and set of tools for managing complex data. It is designed to store large amounts of data efficiently and to support the organization and retrieval of data in a hierarchical structure. HDF5 is widely used in scientific computing, engineering, and data analysis due to its flexibility and performance.

## Key Features of HDF5

1. **Hierarchical Structure**: HDF5 files are organized in a hierarchical structure similar to a file system. This structure consists of groups and datasets:
    - **Groups**: Containers that can hold datasets and other groups, similar to directories in a file system.
    - **Datasets**: Multi-dimensional arrays of data, similar to files in a file system.
2. **Metadata**: HDF5 supports the storage of metadata (attributes) alongside the data. Attributes can be attached to groups and datasets to provide additional information about the data.
3. **Efficient Storage**: HDF5 is optimized for storing large datasets efficiently. It supports compression and chunking to reduce file size and improve access speed.
4. **Portability**: HDF5 files are platform-independent, meaning they can be shared and accessed across different operating systems and hardware platforms.
5. **Scalability**: HDF5 can handle very large datasets, making it suitable for high-performance computing and big data applications.

## Basic Concepts

- **File**: The HDF5 file itself, which contains all the data and metadata.
- **Group**: A container within the HDF5 file that can hold datasets and other groups. Groups are used to organize the data hierarchically.
- **Dataset**: A multi-dimensional array of data stored within a group. Datasets are the primary way to store data in HDF5 files.
- **Attribute**: Metadata associated with a group or dataset. Attributes provide additional information about the data.

## Hierarchy of the HDF5 File for PHYS 2150 - FA25

**IMPORTANT: FA25 uses 3-digit cell numbers (125-197) instead of 2-digit numbers.**

Below is the hierarchy of the HDF5 file for PHYS 2150 FA25. This hierarchy should help you understand how the data is organized within the file.

```
/ (Root)
├── Cell### (Group for each cell, where ### is a 3-digit cell number: 125-197)
│   ├── Metadata (Attributes at CELL level)
│   │   ├── Stress Temperature: 55 (°C)
│   │   └── Stress Wavelength: White (white light)
│   ├── Power/ (Group for Power data)
│   │   ├── YYYY_MM_DD/ (Group for each date)
│   │   │   ├── Data (Dataset containing the Power data)
│   │   │   └── Headers (Dataset containing the headers for the Power data)
│   ├── Pixel# (Group for each pixel, where # is 1-8)
│   │   ├── Current/ (Group for Current data)
│   │   │   ├── YYYY_MM_DD/ (Group for each date)
│   │   │   │   ├── Data (Dataset containing the Current data)
│   │   │   │   └── Headers (Dataset containing the headers for the Current data)
│   │   ├── IV/ (Group for IV/JV data)
│   │   │   ├── YYYY_MM_DD/ (Group for each date)
│   │   │   │   ├── Data (Dataset containing the IV data)
│   │   │   │   └── Headers (Dataset containing the headers for the IV data)
```

## FA25 Dataset Specifics

- **Cell Numbers**: 125-197 (3-digit format)
- **Pixels per Cell**: 8 pixels (1-8)
- **Stress Conditions**: All cells stressed at 55°C with white light (stored as 'White')
- **Measurement Period**: September-October 2025
- **Unmasked Pixel Area**: 0.14 cm²

## Accessing HDF5 Files in Python

Python provides the `h5py` library to work with HDF5 files. Here are some basic operations:

### 1. Opening an HDF5 File

```python
import h5py

# Open an HDF5 file in read mode
hdf5_path = 'data_fa25.h5'
with h5py.File(hdf5_path, 'r') as hdf:
    # Access the root group
    root_group = hdf['/']
```

### 2. Accessing Groups and Datasets

```python
# Access a cell group (FA25: 3-digit cell numbers)
cell_group = hdf['Cell129']

# Access a dataset
power_data = cell_group['Power']['2025_09_23']

# Read data from the dataset
data = power_data['Data'][:]
```

### 3. Reading Attributes (FA25: Metadata at CELL level)

```python
# Access a cell group
cell_group = hdf['Cell129']

# Read cell-level attributes
stress_temp = cell_group.attrs['Stress Temperature']  # Returns '55'
stress_wavelength = cell_group.attrs['Stress Wavelength']  # Returns 'White'
```

# Start your work here




In [ ]:
# Start your code here

# **Author Contributions (required)**



1.   List the name of each team member and please describe briefly how each member contributed to the work on the reseach project.


# **Use of AI (required)**



1.   How your team used AI tools: Describe the specific tasks where AI-assisted tools were involved. For example, did AI help with writing, debugging, optimizing, or refining your code? Which components of the analysis did your team use AI on?
2.   When your team used AI tools: Specify at what stage(s) of your programming process you used AI. Was it during initial code development, troubleshooting, etc.?
3.   Which AI tools your team used: Identify the AI tools or platforms your team consulted (e.g., ChatGPT, GitHub Copilot, etc.).
